In [1]:
from transformers import AutoTokenizer ,AutoModelForCausalLM
import re

In [56]:
# Speller and punctuation

import os
import yaml
import torch
from torch import package

torch.hub.download_url_to_file('https://raw.githubusercontent.com/snakers4/silero-models/master/models.yml',
                               'latest_silero_models.yml',
                               progress=False)

with open('latest_silero_models.yml', 'r') as yaml_file:
    models = yaml.load(yaml_file, Loader=yaml.SafeLoader)
model_conf = models.get('te_models').get('latest')

In [57]:
# Prepare punctuation fix for test
model_url = model_conf.get('package')

model_dir = "downloaded_model"
os.makedirs(model_dir, exist_ok=True)
model_path = os.path.join(model_dir, os.path.basename(model_url))

if not os.path.isfile(model_path):
    torch.hub.download_url_to_file(model_url,
                                   model_path,
                                   progress=True)

imp = package.PackageImporter(model_path)
model = imp.load_pickle("te_model", "model")

def apply_te(text, lan='ru'):
    return model.enhance_text(text, lan)

  0%|          | 0.00/87.5M [00:00<?, ?B/s]

In [2]:
# util function to get expected len after tokenizing
def get_length_param(text: str, tokenizer) -> str:
    tokens_count = len(tokenizer.encode(text))
    if tokens_count <= 15:
        len_param = '1'
    elif tokens_count <= 50:
        len_param = '2'
    elif tokens_count <= 256:
        len_param = '3'
    else:
        len_param = '-'
    return len_param

In [3]:
def remove_duplicates(S):
    S = re.sub(r'[a-zA-Z]+', '', S) #Remove english
    S = S.split()
    result = ""
    for subst in S:
        if subst not in result:
            result += subst+" "
    return result.rstrip()

In [4]:
fit_checkpoint = "WarBot"
tokenizer = AutoTokenizer.from_pretrained(fit_checkpoint)
model = AutoModelForCausalLM.from_pretrained(fit_checkpoint)

In [36]:
quote = "Это хорошо, но глядя на ролик, когда ефиопские толпы в Израиле громят машины и нападают на улице на израильтян - задумаешься, куда все движется.\
Особенно запомнилась картина, когда на проезжающий авто набрасывается штук десять негров и бьют его камнями, запрыгивают на капот и крышу, пытаются через лобовик достать парня-водителя. Жутковато. И им это сошло с рук."

In [37]:
# encode the input, add the eos_token and return a tensor in Pytorch
user_inpit_ids = tokenizer.encode(f"|0|{get_length_param(quote, tokenizer)}|" \
                                              + quote + tokenizer.eos_token, return_tensors="pt")

chat_history_ids = user_inpit_ids # To be changed

tokens_count = len(tokenizer.encode(quote))
if tokens_count < 15:
    no_repeat_ngram_size = 2
else:
    no_repeat_ngram_size = 1

output_id = model.generate(
            chat_history_ids,
            num_return_sequences=2, # use for more variants, but have to print [i]
            max_length=300, #512
            no_repeat_ngram_size=no_repeat_ngram_size, #3
            do_sample=True, #True
            top_k=50,#50
            top_p=0.9, #0.9
            temperature = 0.4, # was 0.6, 0 for greedy
            #mask_token_id=tokenizer.mask_token_id,
            eos_token_id=tokenizer.eos_token_id,
            #unk_token_id=tokenizer.unk_token_id,
            pad_token_id=tokenizer.pad_token_id,
            #pad_token_id=tokenizer.eos_token_id,
            #device='cpu'
        )

90


In [8]:
def removeSigns(S):
    last_index = max(S.rfind("."), S.rfind("!"))
    if last_index >= 0:
        S = S[:last_index+1]
    return S

In [63]:
def getResponce():
    response = tokenizer.decode(output_id[0], skip_special_tokens=True)
    response = removeSigns(response)
    response = response.split(quote)[-1] #Remove the Quote
    response = re.sub(r'[^0-9А-Яа-яЁёa-zA-z;., !()-+:?]', '', response) # Clear the response, remains only alpha-numerical values
    response = remove_duplicates(re.sub(r"\d{4,}", "", response)) # Remove the consequent numbers with 4 or more digits
    return response

In [64]:
print("Response:",getResponce())

Response: я не знаю как там было у вас...но вот вам видео из ливана: эти граждане (в плохом смысле слова) просто тупо ломанулись внутрь! сожалению они были безоружны еще вооружены до зубов....и их всех убили..так что лучше сразу стреляли мне хотябы пытались зайти комне под видом друзей..... готов стрелять хоть сейчас....а потом думал может быть таки стоит подумать где мой пистолет дома если вдруг придется применить...как так..подумайте сами господа...... зызыы для тех кто тут оружие, вы поняли меня правильно! спасибо большое. буду знать теперь обязательно кого нибудь пристрелят случае ))) псы!!! вообщем говоря ситуация такая 1 группа была сама хамасниками поэтому должныватся 2 другая часть которые хотели жить вместе(ну скажем каждый себе злобный буратино).


Punctuation:

In [65]:
input_text = getResponce()
output_text = apply_te(input_text, lan='ru')
print(output_text)

Я не знаю, как там было у вас...но вот вам видео из [UNK] Эти граждане (в плохом смысле слова) просто тупо ломанулись внутрь!. Сожалению, они были безоружны еще вооружены до зубов....и их всех убили..так, что лучше сразу стреляли мне, хотябы пытались зайти комне под видом друзей..... готов стрелять хоть сейчас....а Потом думал может быть таки стоит подумать, где мой пистолет дома, если вдруг придется применить...как так..подумайте сами господа...... зызыы для тех, кто тут оружие, вы поняли меня правильно.! спасибо большое. буду знать теперь обязательно кого-нибудь пристрелят случае ))) псы!!! вообщем говоря ситуация. Такая 1. Группа была сама хамасниками, поэтому должныватся 2. Другая часть, которые хотели жить вместе(ну скажем каждый себе злобный буратино)..


Spelling Fix:

In [41]:
from autocorrect import Speller
spell = Speller('ru')
spell_fix_response = spell(getResponce())

In [42]:
print(spell_fix_response)

я не знаю как там было у вас...но вот вам видео из лимана эти граждане плохом смысле слова просто тупо ломанулись внутрь! сожалению они были безоружны еще вооружены до зубов....и их всех убили..так что лучше сразу стреляли мне хотябы пытались зайти камне под видом друзей..... готов стрелять хоть сейчас....а потом думал может быть таки стоит подумать где мой пистолет дома если вдруг придется применить...как так..подумайте сами господа...... кызы для тех кто тут оружие, вы поняли меня правильно! спасибо большое. буду знать теперь обязательно кого нибудь пристрелят случае псы!!! вообщем говоря ситуация такая 1 группа была сама хамасниками поэтому должныватся 2 другая часть которые хотели жить вместе скажем каждый себе злобный буратино.
